In [1]:
import pandas as pd       
import openpyxl       
from selenium import webdriver       
from selenium.webdriver.common.keys import Keys       
from selenium.webdriver import ActionChains       
from bs4 import BeautifulSoup       
from tkinter import filedialog, Tk       
import time       
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.support import expected_conditions as EC       
from selenium.webdriver.common.by import By  
from selenium import webdriver       
import os       
from tqdm import tqdm       
from selenium.common.exceptions import TimeoutException       
from selenium.webdriver.chrome.options import Options       
import matplotlib.font_manager as fm  
import matplotlib.pyplot as plt  
import datetime       
from selenium.webdriver.chrome.service import Service    
from selenium.common.exceptions import UnexpectedAlertPresentException  

url = "https://www.lotteon.com/p/product/LO1690627968?sitmNo=LO1690627968_1690627969&ch_no=100065&ch_dtl_no=1000030&entryPoint=pcs&dp_infw_cd=CHT&NaPm=ct%3Dlji9aaa8%7Cci%3Dd82c30bcefc2e9b0fc68d6d8b50af93318167051%7Ctr%3Dslsl%7Csn%3D1243359%7Chk%3D2a13ba84a305a3268d41bea82565f152954fc5b2"

options = Options()       
options = webdriver.ChromeOptions()       
driver = webdriver.Chrome('./mnt/c/chromedriver.exe', options=options)       
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})   


driver.get(url)

try:     
    
except AttributeError:     
    seller = "확인필요"        
return seller  

IndentationError: expected an indented block after 'try' statement on line 33 (3474478697.py, line 35)

In [37]:
html = driver.page_source       
soup = BeautifulSoup(html, 'html.parser') 

seller = soup.find("div", attrs={"class": "top"}).get_text().strip()[-6:]
seller

' 프리컴퍼니'

In [6]:

import pandas as pd       
import openpyxl       
from selenium import webdriver       
from selenium.webdriver.common.keys import Keys       
from selenium.webdriver import ActionChains       
from bs4 import BeautifulSoup       
from tkinter import filedialog, Tk       
import time       
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.support import expected_conditions as EC       
from selenium.webdriver.common.by import By  
from selenium import webdriver       
import os       
from tqdm import tqdm       
from selenium.common.exceptions import TimeoutException       
from selenium.webdriver.chrome.options import Options       
import matplotlib.font_manager as fm  
import matplotlib.pyplot as plt  
import datetime       
from selenium.webdriver.chrome.service import Service    
from selenium.common.exceptions import UnexpectedAlertPresentException  


def naver(soup) :      
    try :      
        seller = soup.find("span", attrs={"class": "KasFrJs3SA"}).get_text()          
    except AttributeError :     
        seller = "확인필요"             
    return seller
    
def coupang(soup) :     
    try :      
        seller = soup.find("a", attrs={"class": "prod-sale-vendor-name"}).get_text()           
    except AttributeError :     
        seller = "확인필요"      
    return seller   
       
def interpark(soup):              
    try:     
        seller = soup.find("div", attrs={"class": "sellerName"}).get_text()        
    except AttributeError:     
        seller = "확인필요"     
    return seller
  
def auction(soup) :     
    try :     
        seller = soup.find("a", attrs={"class": "link__seller sp_vipgroup--before sp_vipgroup--after"}).get_text()       
    except AttributeError :     
        seller = "확인필요"     
    return seller  
      
def gmarket(soup) :     
    try:     
        seller = soup.find("a", attrs={"class": "link__seller sp_vipgroup--before sp_vipgroup--after"}).get_text()       
    except AttributeError:     
        seller = "확인필요"      
    return seller
 
def ssg(soup):     
    try:     
        seller = soup.find("a", attrs={"class": "cdtl_info_tit_link"}).get_text()     
    except AttributeError:     
        seller = "확인필요"     
    return seller   

def elevenst(soup):     
    try:     
        seller = soup.find("h1", attrs={"class": "c_product_store_title"}).find('a').get_text().strip()     
    except AttributeError:     
        seller = "확인필요"     
    return seller  
      
def wemakeprice(soup):     
    try:     
        seller = soup.find("span", attrs={"class": "store_name"}).get_text()     
    except AttributeError:     
        seller = "확인필요"     
    return seller  

def lotte(soup):     
    try:     
        seller = soup.find("div", attrs={"class": "top"}).get_text().strip()[-6:] 
    except AttributeError:     
        seller = "확인필요"        
    return seller  

def unknown(soup):     
    seller = "*확인필요*"     
    return seller 
    
            
     
dfs = []
            
# 엑셀 파일 읽어오기       
file_path = filedialog.askopenfilename(title="가격지도 파일", defaultextension=".xlsx")       
df_input = pd.read_excel(file_path, header =0)             
            
options = Options()       
options = webdriver.ChromeOptions()       
driver = webdriver.Chrome('./mnt/c/chromedriver.exe', options=options)       
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})       
        
        
for i in tqdm(range(len(df_input))):       
    # 검색할 키워드를 주문번호로 대체하여 url 생성       
    keyword = str(df_input.loc[i, '주문번호'])       
    minimum_price = int(df_input.loc[i, '최소값'])       
    maximum_price = int(df_input.loc[i, '최대값'])   
    filter = str(df_input.loc[i, '필터'])   
    filter2 = str(df_input.loc[i, '필터2'])  
    filter3 = str(df_input.loc[i, '필터3'])  
                
    #시트에서 지도가 추출하기       
    comsmart_standard = int(df_input.loc[i, '지도가'])       
                
    url = f'https://search.shopping.naver.com/search/all?origQuery={keyword}&pagingIndex=1&pagingSize=80&productSet=total&query={keyword}&sort=rel&timestamp=&viewType=list'     
                
    driver.get(url)       
    time.sleep(2)       
                
    search_btns = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/div[1]/div[1]/div[1]/div[1]/a[2]')))       
                
    for btn in search_btns:       
        if btn.text.strip() == "낮은 가격순":       
            btn.click()       
            break       
                
    #최적화 필터 제거하기           
    next_link = WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.subFilter_btn_radio__13PEL')))       
    if 'on' in next_link.get_attribute('class'):       
        next_link.click()       
                    
    time.sleep(3)       
                    
            
    min_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@title='최소가격 입력']")))     
    min_price.clear()     
    min_price.send_keys(minimum_price)     
        
    time.sleep(1)     
        
    st_date = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@title='최대가격 입력']")))     
    st_date.clear()      
    st_date.send_keys(maximum_price)      
        
    time.sleep(1)     
        
    search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='filter_price_srh__D6arg' and @role='button']")))        
    search_button.click()     
        
    time.sleep(1)     
                
    for c in range(0, 20):       
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/20*" + str(c+1) + ");")       
            time.sleep(0.1)       
                    
    time.sleep(3)    

    des_list = []
    link_list = []
    seller_list = []
    prc_list = []
    gap_list = []
    comment = []
    platform_list = []  
    
    html = driver.page_source       
    soup = BeautifulSoup(html, 'html.parser')        
    flatform = ""
    image_list = soup.select("div.product_item__MDtDF")        
            
    for item in image_list:
        for des in item.select("a.product_link__TrAac.linkAnchor"):
            if 'title' not in des.attrs:  # title 속성이 없을 경우
                continue  # 현재 반복을 건너뛰고 다음 요소로 넘어갑니다.
            
            des_text = des['title']  
            des_list.append(des_text)

            link_text = des['href']
            link_list.append(link_text)

            seller = item.select_one("div.product_mall_title__Xer1m > a")
            seller_text = seller.get_text() if seller else None
            seller_list.append(seller_text)

            platform = item.select_one("div.product_mall_title__Xer1m > a")
            platform_name = platform.img['alt'] if platform and platform.img else "네이버"
            platform_list.append(platform_name)

            price = item.select_one("span.price_num__S2p_v")
            if price:
                prc = int(price.get_text().replace("원", "").replace(",", ""))
                prc_list.append(prc)
                gap = comsmart_standard - prc
                gap_list.append(gap)
                comment_text = "안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 " + str(comsmart_standard) + "원으로 현재 업로드하신 금액과는 당사 지도가 대비" + str(gap) + "원 차이가 있으니 판매 단가 수정을 부탁드립니다." 
                comment.append(comment_text)  
            else:
                prc_list.append(None)
                gap_list.append(None)
                comment.append(None) 

    df = pd.DataFrame({  
        "모델명" : [keyword]*len(des_list),               
        "상세정보": des_list,        
        "지도가" : [comsmart_standard]*len(des_list),          
        "업체등록가": prc_list,      
        "가격차이": gap_list,  
        "판매처" : seller_list,  
        "플랫폼" : platform_list,
        "링크주소": link_list,
        "안내문구" : comment,   
        })     
    
    try :
        if filter :
            df = df[~df['상세정보'].str.contains(filter)]
    except AttributeError :
        pass

    try :
        if filter2 :
            df = df[~df['상세정보'].str.contains(filter2)]
    except AttributeError :
        pass

    try :
        if filter3 :
            df = df[~df['상세정보'].str.contains(filter3)]


    except AttributeError :
        pass
    
    dfs.append(df.copy())
df_total = pd.concat(dfs, ignore_index=True)
df_total = df_total[ df_total['판매처'] != '쇼핑몰별 최저가' ]
df_total = df_total[ df_total['플랫폼'] != '유닛808' ]
df_total = df_total[ df_total['플랫폼'] != 'aliexpress' ]
df_total = df_total[ df_total['플랫폼'] != '교보핫트랙스' ]
df_total = df_total[ df_total['플랫폼'] != '프리쉽' ]
ddf_total = df_total[df_total['가격차이'] >= 1].reset_index(drop=True)
try : 
    for i, row in ddf_total.iterrows():
        if row['판매처'] == "":
            driver.get(row['링크주소'])
            time.sleep(5)       
            html = driver.page_source       
            soup = BeautifulSoup(html, 'lxml')                                                           
                    
            if "coupang" in html :       
                seller = coupang(soup)     
                    
                    
            elif '인터파크쇼핑' in html :                                 
                seller = interpark(soup)     
                    
                    
            elif "G마켓" in html :       
                seller = gmarket(soup)     
                    
                    
            elif "옥션" in html :    
                seller = auction(soup)     
                    
                    
            elif "ssg.com" in html :    
                seller = ssg(soup)     
                    
            elif "11번가" in html :        
                seller = elevenst(soup)     
                    
                    
            elif "위메프" in html :        
                seller = wemakeprice(soup)     
                    
            elif "lotte" in html :       
                seller = lotte(soup)                          
                    
            else :       
                seller = unknown(soup)  
    
            ddf_total.loc[i,'판매처'] = seller 
            
            
except AttributeError:       
    pass       
    
except TypeError:       
    pass       
        
except TimeoutException :       
    pass       

except UnexpectedAlertPresentException: 
    pass 



C:\Users\Owner\AppData\Local\Temp\ipykernel_33460\1255408540.py:101: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./mnt/c/chromedriver.exe', options=options)
100%|██████████| 1/1 [00:17<00:00, 17.65s/it]


In [5]:
ddf_total

,모델명,상세정보,지도가,업체등록가,가격차이,판매처,플랫폼,링크주소,안내문구
0,BT773,컴스몰 [BT773] USB 허브 2.0 (7P/무전원) 각스위치,19550,18200,1350,컴스몰,네이버,https://cr.shopping.naver.com/adcr.nhn?x=Ip2mN...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...
1,BT773,Coms 7포트 USB 3.0 허브 무전원 개별스위치 LED표시 BT773,19550,19540,10,리빙꼴레뜨,G마켓,https://cr.shopping.naver.com/adcr.nhn?x=PFCCr...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...


In [18]:
print(len(des_list))
print(len(prc_list))
print(len(gap_list))
print(len(seller_list))
print(len(platform_list))
print(len(link_list))
print(len(comment))


51
51
51
51
51
51
51


In [19]:
ddf_total

,모델명,상세정보,지도가,업체등록가,가격차이,판매처,플랫폼,링크주소,안내문구
0,BT773,컴스몰 [BT773] USB 허브 2.0 (7P/무전원) 각스위치,19550,18200,1350,컴스몰,네이버,https://cr.shopping.naver.com/adcr.nhn?x=pw2Y5...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...
1,BT773,BT773 Coms USB 3.0 7포트 허브 (무전원 각스위치),19550,18370,1180,포유스토어,네이버,https://cr.shopping.naver.com/adcr.nhn?x=L4VIe...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...
2,BT773,coms BT773 USB 3.0 7포트 허브 (무전원 각스위치),19550,18370,1180,만가지샵,네이버,https://cr.shopping.naver.com/adcr.nhn?x=E2%2B...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...
3,BT773,[BT773] Coms USB 3.0 7포트 허브 (무전원 / 각스위치),19550,18740,810,제플몰,네이버,https://cr.shopping.naver.com/adcr.nhn?x=E%2F9...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...
4,BT773,7포트 USB 3.0 허브 무전원 개별스위치 LED표시 BT773,19550,18770,780,리빙꼴레뜨,11번가,https://cr.shopping.naver.com/adcr.nhn?x=2m3IS...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...
5,BT773,[BT773] Coms USB 3.0 7포트 허브 (무전원 / 각스위치),19550,19370,180,네모아이티,네이버,https://cr.shopping.naver.com/adcr.nhn?x=y7sN7...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...
6,BT773,Coms 7포트 USB 3.0 허브 무전원 개별스위치 LED표시 BT773,19550,19540,10,리빙꼴레뜨,G마켓,https://cr.shopping.naver.com/adcr.nhn?x=jAMTC...,안녕하세요. 컴스마트 관리부입니다. 현재 업로드 하신 제품의 당사 지도가는 1955...


In [13]:
for i in image_list :
    for e in i.select("a.product_link__TrAac.linkAnchor") :
        제품정보 = e.get_text()
        print(제품정보)

컴스몰 [BT773] USB 허브 2.0 (7P/무전원) 각스위치
BT773 Coms USB 3.0 7포트 허브 (무전원 각스위치)
coms BT773 USB 3.0 7포트 허브 (무전원 각스위치)
[BT773] Coms USB 3.0 7포트 허브 (무전원 / 각스위치)
7포트 USB 3.0 허브 무전원 개별스위치 LED표시 BT773
[BT773] Coms USB 3.0 7포트 허브 (무전원 / 각스위치)
라이트컴 COMS 7포트 USB 3.0 허브 (BT773)
기본구성
9개
무전원(USB)
19개
Coms 7포트 USB 3.0 허브 무전원 개별스위치 LED표시 BT773
Coms BT773 USB 허브 3.0 (7P/무전원) - 개별스위치
COMS BT773 USB허브 3.0 7포트 무전원
BT773 Coms USB 3.0 7포트 허브 무전원 각
BT773 Coms USB 3.0 7포트 허브 무전원 각
Coms USB 3.0 7포트 USB허브 무전원 BT773
BT773 Coms USB 3.0 7포트 허브 무전원 각
BT773 Coms USB 3.0 7포트 허브 무전원 각
Coms BT773 USB 허브 3.0 (7P/무전원) - 개별스위치
라이트컴 7포트 USB2.0 허브 BT773
마트천 Coms USB 3.0 7포트 허브 BT773
BT773 Coms USB 3.0 7포트 허브 무전원 각
BT773 Coms USB 3.0 7포트 허브 무전원 각
Coms USB 허브 3.0 7P/무전원-각스위치 BT773
Coms BT773 USB 허브 3.0 (7P/무전원) - 각스위치
Coms USB 3.0 7포트 허브 BT773
BT773 Coms USB 3.0 7포트 허브 무전원 각
BT773 Coms USB 3.0 7포트 허브 무전원 각
BT773 Coms USB 3.0 7포트 허브 무전원 각
USB 3.0 7포트 허브 (무전원 / 각스위치) BT773
BT773 Coms USB 3.0 7포트 허브 무전원 각
(COMS) USB 3.0 7포트 무전원허브(스위

In [24]:
ddf_total.to_excel("c://excel//result16.xlsx")